In [1]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [2]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "medical_history").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [3]:
from medminer.task.history import history_task

for doc in docs:
    history_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: history                                                                                              │
│ Prompt:                                                                                                         │
│     Given a medical history of a patient, extract all given diagnoses and save all information as csv. The      │
│ diagnosis should be translated to english. The medical history is usually in the format of a sentence or a      │
│ paragraph. Every diagnosis should have a single row, if there are multiple diagnosis that can be extracted from │
│ a single piece of text, split them up. These are the steps you should follow to complete the task:              │
│     1. extract a part of the text that contains a diagnosis. The diagnosis can be in any language. This is the  │
│ diagnosis_reference column.                                                                                     │
│     2. translate the diagnoses to english if necessary and infer the diagnosis_translated column.               │
│     3. Extract the relevant diagnosis as a string and loose everything that is not relevant. This is the        │
│ diagnosis column.                                                                                               │
│     4. Extract the month and year of the medical history for that diagnosis. If not applicable, write an empty  │
│ string.                                                                                                         │
│                                                                                                                 │
│     Example 1:                                                                                                  │
│     Input: "schwere Herzinsuffizienz, 2. Grad, NYHA II, ED 07/2023"                                             │
│     Output: [{"patient_id": 1, "diagnosis_reference": "schwere chronische Herzinsuffizienz, 2. Grad, NYHA II",  │
│ "diagnosis_translated": "severe chronic heart failure, 2nd degree, NYHA II", "diagnosis": "chronic heart        │
│ failure", "month": "7", "year": "2023"}\]                                                                       │
│                                                                                                                 │
│     Example 2:                                                                                                  │
│     Input: "Myocardial Infarction"                                                                              │
│     Output: [{"patient_id": 2, "diagnosis_reference": "Myocardial Infarction", "diagnosis_translated":          │
│ "Myocardial Infarction", "diagnosis": "Myocardial Infarction", "month": "", "year": ""}\]                       │
│                                                                                                                 │
│     Example 3:                                                                                                  │
│     Input: "Lungenteilresektion rechts bei Lungenmetastasen eines Kolonkarzinoms, 2023-07-01"                   │
│     Output: [                                                                                                   │
│         {"patient_id": 3, "diagnosis_reference": "Lungenteilresektion rechts bei Lungenmetastasen eines         │
│ Kolonkarzinoms", "diagnosis_translated": "Lung resection due to lung metastases of colon cancer", "diagnosis":  │
│ "colon cancer", "month": "7", "year": "2023"},                                                                  │
│         {"patient_id": 3, "diagnosis_reference": "Lungenteilresektion rechts bei Lungenmetastasen eines         │
│ Kolonkarzinoms", "diagnosis_translated": "Lung resecti

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'save_csv' with arguments: {'task_name': 'history', 'data': [{'patient_id': 1,                    │
│ 'diagnosis_reference': 'SolitÃ¤re Lungenmetastase Oberlappen bei Adeno-CA des Kolons ED 2020',                  │
│ 'diagnosis_translated': 'Solitary lung metastasis upper lobe due to adenocarcinoma of the colon', 'diagnosis':  │
│ 'adenocarcinoma of the colon', 'month': '', 'year': '2020'}, {'patient_id': 1, 'diagnosis_reference': 'Z. n.    │
│ Resektion einer weiteren Unterlappenmetastase 2021', 'diagnosis_translated': 'Status post resection of another  │
│ lower lobe metastasis', 'diagnosis': 'lower lobe metastasis', 'month': '', 'year': '2021'}, {'patient_id': 1,   │
│ 'diagnosis_reference': 'Z. n. Lungenteilresektion rechts im November 2018 in Köln', 'diagnosis_translated':     │
│ 'Status post partial lung resection on the right in November 2018 in Cologne', 'diagnosis': 'partial lung       │
│ resection', 'month': '11', 'year': '2018'}, {'patient_id': 1, 'diagnosis_reference': 'Z. n.                     │
│ Radio-/Chemotherapie bis Herbst 2022', 'diagnosis_translated': 'Status post radiotherapy/chemotherapy until     │
│ autumn 2022', 'diagnosis': 'radiotherapy/chemotherapy', 'month': '', 'year': '2022'}, {'patient_id': 1,         │
│ 'diagnosis_reference': 'Z. n. Appendektomie', 'diagnosis_translated': 'Status post appendectomy', 'diagnosis':  │
│ 'appendectomy', 'month': '', 'year': ''}, {'patient_id': 1, 'diagnosis_reference': 'Z. n. Hemikolektomie links  │
│ bei ausgeprägter Sigmadivertikulitis', 'diagnosis_translated': 'Status post left hemicolectomy due to severe    │
│ sigmoid diverticulitis', 'diagnosis': 'sigmoid diverticulitis', 'month': '', 'year': ''}, {'patient_id': 1,     │
│ 'diagnosis_reference': 'unklare RF Milz, a.e. benigne', 'diagnosis_translated': 'Unclear spleen lesion,         │
│ probably benign', 'diagnosis': 'benign spleen lesion', 'month': '', 'year': ''}, {'patient_id': 1,              │
│ 'diagnosis_reference': 'Z. n Cholezystektomie bei Cholezystits 2000', 'diagnosis_translated': 'Status post      │
│ cholecystectomy due to cholecystitis 2000', 'diagnosis': 'cholecystitis', 'month': '', 'year': '2000'},         │
│ {'patient_id': 1, 'diagnosis_reference': 'Gastritis', 'diagnosis_translated': 'Gastritis', 'diagnosis':         │
│ 'Gastritis', 'month': '', 'year': ''}]}                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Saved data for task history to c:\Users\chris\Projects\aidh-ms\MedMiner\examples\result\history.csv

[Step 1: Duration 5.50 seconds| Input tokens: 1,847 | Output tokens: 532]

...

[Step 1: Duration 9.17 seconds| Input tokens: 1,780 | Output tokens: 407]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The diagnoses from the medical history have been       │
│ successfully extracted and saved to a CSV file at                                                               │
│ 'c:\\Users\\chris\\Projects\\aidh-ms\\MedMiner\\examples\\result\\history.csv'."}                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The diagnoses from the medical history have been successfully extracted and saved to a CSV file at 
'c:\Users\chris\Projects\aidh-ms\MedMiner\examples\result\history.csv'.

[Step 2: Duration 0.86 seconds| Input tokens: 4,133 | Output tokens: 460]